<a href="https://colab.research.google.com/github/adithya72/Deep-Learning/blob/Develop/SingalQualityPredictionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Networks - Predict equipment signal quality

A communications equipment manufacturing company has a product which is responsible for emitting informative signals. Company wants to build a machine learning model which can help the company to predict the equipment’s signal quality using various parameters.

 **Objective** : Build a classifier neural network model that can take various singal parameters as inputs and determine/predict the signal strength or quality.

**Dataset** : The data set contains information on various signal tests performed:
1. Parameters: Various measurable signal parameters.
2. Signal_Quality: Final signal strength or quality

**ML libraries** : Keras, tensorflow

This notebook covers,

1. Data importing and understanding
2. Data preprocessing
3. Model Training & Evaluation using Neural Network


# 1. Data importing and understanding

In [ ]:
#import required libraries
import os
import pandas as pd
import numpy as np
import tensorflow


In [ ]:
# Mount google drive where thed dataset file resides
from google.colab import drive
drive.mount("/content/Drive")
os.getcwd() #check the current working directory

Mounted at /content/Drive


'/content'

In [ ]:
#import the dataset - signal.csv from google drive
dataset_file = '/content/Drive/MyDrive/NN Dataset/Signal.csv'
signaldata = pd.read_csv(dataset_file)
print("Total numbers of rows and columns in dataset : ", signaldata.shape)
print("Glance of first few rows")
signaldata.head()


Total numbers of rows and columns in dataset :  (1599, 12)
Glance of first few rows


,Parameter 1,Parameter 2,Parameter 3,Parameter 4,Parameter 5,Parameter 6,Parameter 7,Parameter 8,Parameter 9,Parameter 10,Parameter 11,Signal_Strength
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
#Check for missing values and print percentage for each attribute.
# The percentage of missing values in a dataset is an informative measure of how much data is missing or unavailable.
# It can help assess the quality and reliability of the data, and to decide how to handle the missing values.

# Calculate the number of missing values in each column and print
missing_values = signaldata.isnull().sum()
print("1. Missing values in each column")
print(missing_values)

# Calculate the total number of values in each column
total_values = signaldata.count() + missing_values
print("2. Total number of values in each column")
print(total_values)

# Calculate the percentage of missing values in each column
percentage_missing = (missing_values / total_values) * 100

# Print the results
print("3. percentage of missing value")
print(percentage_missing)


1. Missing values in each column
Parameter 1        0
Parameter 2        0
Parameter 3        0
Parameter 4        0
Parameter 5        0
Parameter 6        0
Parameter 7        0
Parameter 8        0
Parameter 9        0
Parameter 10       0
Parameter 11       0
Signal_Strength    0
dtype: int64
2. Total number of values in each column
Parameter 1        1599
Parameter 2        1599
Parameter 3        1599
Parameter 4        1599
Parameter 5        1599
Parameter 6        1599
Parameter 7        1599
Parameter 8        1599
Parameter 9        1599
Parameter 10       1599
Parameter 11       1599
Signal_Strength    1599
dtype: int64
3. percentage of missing value
Parameter 1        0.0
Parameter 2        0.0
Parameter 3        0.0
Parameter 4        0.0
Parameter 5        0.0
Parameter 6        0.0
Parameter 7        0.0
Parameter 8        0.0
Parameter 9        0.0
Parameter 10       0.0
Parameter 11       0.0
Signal_Strength    0.0
dtype: float64


Conclusion - Zero missing values

In [ ]:
# Check for presence of duplicate records in the dataset and impute with appropriate method.
# duplicate records can be problematic because they can skew the results

#Check for duplicate records
signaldata.duplicated().sum()


240

Conclusion - There are 240 duplicate records, hence we should be dropping them

In [ ]:
signaldata_withoutDup = signaldata.drop_duplicates()
print("Total number of rows and columns with out duplicate data",signaldata_withoutDup.shape)

Total number of rows and columns with out duplicate data (1359, 12)


In [ ]:
#Visualise distribution of the target variable.
#Understading each varible is important

